# Train a model to recogise static gestures

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Prepare the data

In [2]:
import numpy as np
import pandas as pd
import pickle
import os

In [3]:
data = pickle.load(open('../out_data/robot_hand_keypoints_120421_03_26.pkl', 'rb'))
data

[{'hand_type': 'right',
  'gesture_id': 0,
  'gesture_name': 'no_gesture',
  'pred_gesture_name': 'FIVE',
  'keypoints': array([[ 2.61616558e-01,  8.62391472e-01, -1.18615622e-04],
         [ 3.40141207e-01,  8.32768440e-01, -3.68560776e-02],
         [ 3.95898253e-01,  7.64145434e-01, -5.05490154e-02],
         [ 4.23794150e-01,  6.91991389e-01, -6.58366755e-02],
         [ 4.54131007e-01,  6.34845853e-01, -8.54506791e-02],
         [ 3.52733880e-01,  6.40689015e-01, -3.90581018e-03],
         [ 3.70479733e-01,  5.49144268e-01, -1.90667845e-02],
         [ 3.82825851e-01,  4.93496358e-01, -3.28761302e-02],
         [ 3.93023401e-01,  4.43832815e-01, -4.70148660e-02],
         [ 3.07039738e-01,  6.26214385e-01, -2.82135443e-03],
         [ 3.11871171e-01,  5.25952458e-01, -1.28231077e-02],
         [ 3.18490505e-01,  4.61563885e-01, -2.94405650e-02],
         [ 3.22950721e-01,  4.09772158e-01, -4.30353209e-02],
         [ 2.64687926e-01,  6.30137026e-01, -1.12086385e-02],
         [ 2.

> **Note:** The data is collected for the right hand only.

In [4]:
gesture_ids = np.array([x['gesture_id'] for x in data]).reshape(-1, 1)
gesture_names = np.array([x['gesture_name'] for x in data]).reshape(-1, 1)
keypoints = np.array([x['keypoints'] for x in data]).reshape(-1, 63)

In [5]:
print(gesture_ids.shape, gesture_names.shape, keypoints.shape)

(1901, 1) (1901, 1) (1901, 63)


In [6]:
columns = ['gesture_id', 'gesture_name']
for i in range(keypoints.shape[1] // 3):
    columns += [f'kp{i}_x', f'kp{i}_y', f'kp{i}_z']
df = pd.DataFrame(data=np.hstack([gesture_ids, gesture_names, keypoints]), 
                  columns=columns)
df

,gesture_id,gesture_name,kp0_x,kp0_y,kp0_z,kp1_x,kp1_y,kp1_z,kp2_x,kp2_y,...,kp17_z,kp18_x,kp18_y,kp18_z,kp19_x,kp19_y,kp19_z,kp20_x,kp20_y,kp20_z
0,0,no_gesture,0.2616165578365326,0.862391471862793,-0.00011861562234116718,0.3401412069797516,0.832768440246582,-0.03685607761144638,0.3958982527256012,0.7641454339027405,...,-0.024227840825915337,0.21081572771072388,0.5820988416671753,-0.036938998848199844,0.2077687680721283,0.5350749492645264,-0.05315976217389107,0.20487046241760254,0.4915812313556671,-0.06644965708255768
1,0,no_gesture,0.2488212138414383,0.8327491879463196,-8.602064190199599e-05,0.3300294280052185,0.8062025308609009,-0.05029881373047829,0.3931019902229309,0.7401115298271179,...,-0.05144594609737396,0.18782782554626465,0.5499157309532166,-0.07304853200912476,0.178327739238739,0.5024149417877197,-0.09637845307588577,0.16922423243522644,0.4561491012573242,-0.11695774644613266
2,0,no_gesture,0.2098255455493927,0.806172788143158,-7.33856504666619e-05,0.293041467666626,0.8084140419960022,-0.035692423582077026,0.36657994985580444,0.7661011219024658,...,-0.03616072237491608,0.23157352209091187,0.5067298412322998,-0.056251414120197296,0.2460532784461975,0.4627677798271179,-0.08015701919794083,0.26056623458862305,0.42247650027275085,-0.09676968306303024
3,0,no_gesture,0.1882975697517395,0.796644926071167,-4.8352027079090476e-05,0.27230411767959595,0.8144708871841431,-0.03367951884865761,0.3501909077167511,0.7865704894065857,...,-0.022155076265335083,0.25076210498809814,0.5014303922653198,-0.03654671460390091,0.27215859293937683,0.46206140518188477,-0.05788211151957512,0.29213279485702515,0.42708343267440796,-0.07161156088113785
4,0,no_gesture,0.1724901795387268,0.7742319703102112,-6.687969289487228e-05,0.2561293840408325,0.804787814617157,-0.018885411322116852,0.3348514437675476,0.7911533117294312,...,-0.015632126480340958,0.27180129289627075,0.4929482340812683,-0.02364089898765087,0.298082560300827,0.4571201205253601,-0.041698750108480453,0.3216680884361267,0.4257204532623291,-0.053536172956228256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,0,no_gesture,0.32307901978492737,0.6458818912506104,-2.1450407075462863e-05,0.365364134311676,0.7069713473320007,-0.01946178823709488,0.4299249053001404,0.737430214881897,...,-0.029674027115106583,0.4608999192714691,0.4699644148349762,-0.06489071995019913,0.4925169050693512,0.44605037569999695,-0.08293934166431427,0.5246965885162354,0.41828328371047974,-0.09062712639570236
1897,0,no_gesture,0.3506574034690857,0.7074780464172363,-6.25709944870323e-05,0.4089662432670593,0.7137359380722046,-0.028144683688879013,0.47107112407684326,0.6655962467193604,...,-0.056962061673402786,0.3344624638557434,0.4835580587387085,-0.09510712325572968,0.3327365517616272,0.4387051463127136,-0.11736176162958145,0.33124879002571106,0.38620853424072266,-0.13155074417591095
1898,0,no_gesture,0.3773188591003418,0.7767844200134277,-1.5660458302590996e-05,0.42634764313697815,0.7357620000839233,-0.046103738248348236,0.4525631368160248,0.6489238739013672,...,-0.046092595905065536,0.270196795463562,0.6188383102416992,-0.0734802708029747,0.24736957252025604,0.5900310277938843,-0.08739117532968521,0.2180003523826599,0.5517174601554871,-0.09553233534097672
1899,0,no_gesture,0.3849989175796509,0.8331529498100281,-4.887472096015699e-05,0.423456609249115,0.7704190611839294,-0.02924411930143833,0.42183586955070496,0.6738439202308655,...,-0.05820860713720322,0.23991179466247559,0.7386565208435059,-0.09852148592472076,0.21134266257286072,0.7226414680480957,-0.11997214704751968,0.17695461213588715,0.7038993239402771,-0.1325664520263672


In [7]:
df['gesture_name'].value_counts()

no_gesture    995
angle         306
move          300
grab          300
Name: gesture_name, dtype: int64

## Train sklearn model

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score

In [9]:
X, y = df.iloc[:, 2:].values.astype('float'), df.iloc[:, 0].values.astype('int32')
print(X.shape, y.shape)
print(X[0], y[0], np.unique(y))

(1901, 63) (1901,)
[ 2.61616558e-01  8.62391472e-01 -1.18615622e-04  3.40141207e-01
  8.32768440e-01 -3.68560776e-02  3.95898253e-01  7.64145434e-01
 -5.05490154e-02  4.23794150e-01  6.91991389e-01 -6.58366755e-02
  4.54131007e-01  6.34845853e-01 -8.54506791e-02  3.52733880e-01
  6.40689015e-01 -3.90581018e-03  3.70479733e-01  5.49144268e-01
 -1.90667845e-02  3.82825851e-01  4.93496358e-01 -3.28761302e-02
  3.93023401e-01  4.43832815e-01 -4.70148660e-02  3.07039738e-01
  6.26214385e-01 -2.82135443e-03  3.11871171e-01  5.25952458e-01
 -1.28231077e-02  3.18490505e-01  4.61563885e-01 -2.94405650e-02
  3.22950721e-01  4.09772158e-01 -4.30353209e-02  2.64687926e-01
  6.30137026e-01 -1.12086385e-02  2.63029665e-01  5.37684083e-01
 -2.10358277e-02  2.67169356e-01  4.76384193e-01 -3.83302718e-02
  2.70671397e-01  4.25496340e-01 -5.44468910e-02  2.22248927e-01
  6.51061773e-01 -2.42278408e-02  2.10815728e-01  5.82098842e-01
 -3.69389988e-02  2.07768768e-01  5.35074949e-01 -5.31597622e-02
  2.04

In [10]:
idx = np.arange(X.shape[0])
np.random.shuffle(idx)
X, y = X[idx], y[idx]

In [11]:
class_amounts = {0: 400, 1: 200, 2: 200, 3: 200}

X_new, y_new = None, None
for class_id in np.unique(y):
    class_indices = np.argwhere(class_id == y)
    print(class_indices.shape)
    subsample_indices = np.random.choice(class_indices[:,0], size=class_amounts[class_id])
    print(subsample_indices.shape)
    if X_new is None:
        X_new = X[subsample_indices]
    else:
        X_new = np.vstack((X_new, X[subsample_indices]))
    if y_new is None:
        y_new = y[subsample_indices].reshape(-1,1)
    else:
        y_new = np.vstack((y_new, y[subsample_indices].reshape(-1,1)))

print(X_new.shape, y_new.shape)
X = X_new
y = y_new

(995, 1)
(400,)
(300, 1)
(200,)
(306, 1)
(200,)
(300, 1)
(200,)
(1000, 63) (1000, 1)


In [12]:
logreg = LogisticRegression()
cross_val_score(logreg, X=X, y=y, scoring=make_scorer(accuracy_score), cv=5)

array([0.86, 0.86, 0.86, 0.86, 0.89])

In [13]:
rfclf = RandomForestClassifier()
cross_val_score(rfclf, X=X, y=y, scoring=make_scorer(accuracy_score), cv=5)

array([0.9  , 0.915, 0.965, 0.945, 0.955])

In [14]:
gbclf = GradientBoostingClassifier()
cross_val_score(gbclf, X=X, y=y, scoring=make_scorer(accuracy_score), cv=5)

array([0.925, 0.91 , 0.975, 0.95 , 0.97 ])

In [15]:
models_root = './robot_models'
os.makedirs(models_root, exist_ok=True)
pickle.dump(gbclf, open(f'{models_root}/robot_gb_default.pkl', 'wb'))

## Train Keras model

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [36]:
X, y = df.iloc[:, 2:].values.astype('float'), df.iloc[:, 0].values.astype('int32')
print(X.shape, y.shape)
print(X[0], y[0], np.unique(y))

(1901, 63) (1901,)
[ 2.61616558e-01  8.62391472e-01 -1.18615622e-04  3.40141207e-01
  8.32768440e-01 -3.68560776e-02  3.95898253e-01  7.64145434e-01
 -5.05490154e-02  4.23794150e-01  6.91991389e-01 -6.58366755e-02
  4.54131007e-01  6.34845853e-01 -8.54506791e-02  3.52733880e-01
  6.40689015e-01 -3.90581018e-03  3.70479733e-01  5.49144268e-01
 -1.90667845e-02  3.82825851e-01  4.93496358e-01 -3.28761302e-02
  3.93023401e-01  4.43832815e-01 -4.70148660e-02  3.07039738e-01
  6.26214385e-01 -2.82135443e-03  3.11871171e-01  5.25952458e-01
 -1.28231077e-02  3.18490505e-01  4.61563885e-01 -2.94405650e-02
  3.22950721e-01  4.09772158e-01 -4.30353209e-02  2.64687926e-01
  6.30137026e-01 -1.12086385e-02  2.63029665e-01  5.37684083e-01
 -2.10358277e-02  2.67169356e-01  4.76384193e-01 -3.83302718e-02
  2.70671397e-01  4.25496340e-01 -5.44468910e-02  2.22248927e-01
  6.51061773e-01 -2.42278408e-02  2.10815728e-01  5.82098842e-01
 -3.69389988e-02  2.07768768e-01  5.35074949e-01 -5.31597622e-02
  2.04

In [37]:
idx = np.arange(X.shape[0])
np.random.shuffle(idx)
X, y = X[idx], y[idx]

In [38]:
n_classes = len(np.unique(y))
n_feats = X.shape[1]

In [39]:
# class_amounts = {0: 400, 1: 200, 2: 200, 3: 200}

# X_new, y_new = None, None
# for class_id in np.unique(y):
#     class_indices = np.argwhere(class_id == y)
#     print(class_indices.shape)
#     subsample_indices = np.random.choice(class_indices[:,0], size=class_amounts[class_id])
#     print(subsample_indices.shape)
#     if X_new is None:
#         X_new = X[subsample_indices]
#     else:
#         X_new = np.vstack((X_new, X[subsample_indices]))
#     if y_new is None:
#         y_new = y[subsample_indices].reshape(-1,1)
#     else:
#         y_new = np.vstack((y_new, y[subsample_indices].reshape(-1,1)))

# print(X_new.shape, y_new.shape)
# X = X_new
# y = y_new

In [40]:
y_ohe = np.zeros((y.shape[0], n_classes))
y_ohe[np.arange(len(y)).reshape(-1,1), y.reshape(-1, 1)] = 1
print(y_ohe.sum(axis=0))

[995. 300. 306. 300.]


In [41]:
train_ratio = 0.75
X_train, y_train = X[:int(len(X)*train_ratio)], y_ohe[:int(len(X)*train_ratio)]
X_val, y_val = X[int(len(X)*train_ratio):], y[int(len(X)*train_ratio):]

* One neuron (linear / logistic regression):

In [42]:
loss_func = keras.losses.CategoricalCrossentropy(from_logits=True)

lr = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

model = keras.Sequential([keras.layers.Dense(units=n_classes, input_shape=[n_feats])])
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
45/45 [==============================] - 0s 647us/step - loss: 1.7390 - accuracy: 0.1565
Epoch 2/500
45/45 [==============================] - 0s 678us/step - loss: 1.6630 - accuracy: 0.1565
Epoch 3/500
45/45 [==============================] - 0s 673us/step - loss: 1.5959 - accuracy: 0.1565
Epoch 4/500
45/45 [==============================] - 0s 666us/step - loss: 1.5364 - accuracy: 0.1565
Epoch 5/500
45/45 [==============================] - 0s 673us/step - loss: 1.4860 - accuracy: 0.1565
Epoch 6/500
45/45 [==============================] - 0s 670us/step - loss: 1.4422 - accuracy: 0.1593
Epoch 7/500
45/45 [==============================] - 0s 643us/step - loss: 1.4060 - accuracy: 0.1719
Epoch 8/500
45/45 [==============================] - 0s 682us/step - loss: 1.3752 - accuracy: 0.2765
Epoch 9/500
45/45 [==============================] - 0s 898us/step - loss: 1.3504 - accuracy: 0.4611
Epoch 10/500
45/45 [==============================] - 0s 676us/step - loss: 1.3296 - accura

45/45 [==============================] - 0s 709us/step - loss: 1.1835 - accuracy: 0.5235
Epoch 82/500
45/45 [==============================] - 0s 665us/step - loss: 1.1824 - accuracy: 0.5235
Epoch 83/500
45/45 [==============================] - 0s 644us/step - loss: 1.1813 - accuracy: 0.5235
Epoch 84/500
45/45 [==============================] - 0s 649us/step - loss: 1.1803 - accuracy: 0.5235
Epoch 85/500
45/45 [==============================] - 0s 671us/step - loss: 1.1792 - accuracy: 0.5235
Epoch 86/500
45/45 [==============================] - 0s 658us/step - loss: 1.1781 - accuracy: 0.5235
Epoch 87/500
45/45 [==============================] - 0s 654us/step - loss: 1.1770 - accuracy: 0.5235
Epoch 88/500
45/45 [==============================] - 0s 639us/step - loss: 1.1760 - accuracy: 0.5235
Epoch 89/500
45/45 [==============================] - 0s 655us/step - loss: 1.1749 - accuracy: 0.5235
Epoch 90/500
45/45 [==============================] - 0s 657us/step - loss: 1.1738 - accuracy: 

45/45 [==============================] - 0s 687us/step - loss: 1.1072 - accuracy: 0.5270
Epoch 161/500
45/45 [==============================] - 0s 777us/step - loss: 1.1064 - accuracy: 0.5270
Epoch 162/500
45/45 [==============================] - 0s 872us/step - loss: 1.1056 - accuracy: 0.5270
Epoch 163/500
45/45 [==============================] - 0s 697us/step - loss: 1.1046 - accuracy: 0.5270
Epoch 164/500
45/45 [==============================] - 0s 665us/step - loss: 1.1038 - accuracy: 0.5270
Epoch 165/500
45/45 [==============================] - 0s 664us/step - loss: 1.1030 - accuracy: 0.5270
Epoch 166/500
45/45 [==============================] - 0s 725us/step - loss: 1.1024 - accuracy: 0.5270
Epoch 167/500
45/45 [==============================] - 0s 760us/step - loss: 1.1014 - accuracy: 0.5270
Epoch 168/500
45/45 [==============================] - 0s 1ms/step - loss: 1.1006 - accuracy: 0.5277
Epoch 169/500
45/45 [==============================] - 0s 740us/step - loss: 1.0997 - acc

45/45 [==============================] - 0s 737us/step - loss: 1.0469 - accuracy: 0.5389
Epoch 240/500
45/45 [==============================] - 0s 774us/step - loss: 1.0461 - accuracy: 0.5389
Epoch 241/500
45/45 [==============================] - 0s 771us/step - loss: 1.0456 - accuracy: 0.5389
Epoch 242/500
45/45 [==============================] - 0s 759us/step - loss: 1.0448 - accuracy: 0.5396
Epoch 243/500
45/45 [==============================] - 0s 761us/step - loss: 1.0440 - accuracy: 0.5396
Epoch 244/500
45/45 [==============================] - 0s 751us/step - loss: 1.0436 - accuracy: 0.5396
Epoch 245/500
45/45 [==============================] - 0s 701us/step - loss: 1.0427 - accuracy: 0.5418
Epoch 246/500
45/45 [==============================] - 0s 661us/step - loss: 1.0421 - accuracy: 0.5432
Epoch 247/500
45/45 [==============================] - 0s 656us/step - loss: 1.0413 - accuracy: 0.5411
Epoch 248/500
45/45 [==============================] - 0s 643us/step - loss: 1.0406 - a

45/45 [==============================] - 0s 639us/step - loss: 0.9964 - accuracy: 0.5607
Epoch 319/500
45/45 [==============================] - 0s 642us/step - loss: 0.9959 - accuracy: 0.5607
Epoch 320/500
45/45 [==============================] - 0s 657us/step - loss: 0.9953 - accuracy: 0.5607
Epoch 321/500
45/45 [==============================] - 0s 633us/step - loss: 0.9948 - accuracy: 0.5586
Epoch 322/500
45/45 [==============================] - 0s 622us/step - loss: 0.9941 - accuracy: 0.5607
Epoch 323/500
45/45 [==============================] - 0s 649us/step - loss: 0.9936 - accuracy: 0.5607
Epoch 324/500
45/45 [==============================] - 0s 656us/step - loss: 0.9929 - accuracy: 0.5607
Epoch 325/500
45/45 [==============================] - 0s 665us/step - loss: 0.9924 - accuracy: 0.5607
Epoch 326/500
45/45 [==============================] - 0s 662us/step - loss: 0.9917 - accuracy: 0.5607
Epoch 327/500
45/45 [==============================] - 0s 649us/step - loss: 0.9915 - a

45/45 [==============================] - 0s 638us/step - loss: 0.9526 - accuracy: 0.5811
Epoch 398/500
45/45 [==============================] - 0s 639us/step - loss: 0.9523 - accuracy: 0.5811
Epoch 399/500
45/45 [==============================] - 0s 840us/step - loss: 0.9516 - accuracy: 0.5811
Epoch 400/500
45/45 [==============================] - 0s 729us/step - loss: 0.9510 - accuracy: 0.5811
Epoch 401/500
45/45 [==============================] - 0s 658us/step - loss: 0.9505 - accuracy: 0.5818
Epoch 402/500
45/45 [==============================] - 0s 699us/step - loss: 0.9500 - accuracy: 0.5832
Epoch 403/500
45/45 [==============================] - 0s 708us/step - loss: 0.9495 - accuracy: 0.5832
Epoch 404/500
45/45 [==============================] - 0s 829us/step - loss: 0.9489 - accuracy: 0.5825
Epoch 405/500
45/45 [==============================] - 0s 787us/step - loss: 0.9484 - accuracy: 0.5832
Epoch 406/500
45/45 [==============================] - 0s 816us/step - loss: 0.9479 - a

45/45 [==============================] - 0s 636us/step - loss: 0.9131 - accuracy: 0.6000
Epoch 477/500
45/45 [==============================] - 0s 655us/step - loss: 0.9125 - accuracy: 0.6000
Epoch 478/500
45/45 [==============================] - 0s 648us/step - loss: 0.9120 - accuracy: 0.6007
Epoch 479/500
45/45 [==============================] - 0s 633us/step - loss: 0.9114 - accuracy: 0.6000
Epoch 480/500
45/45 [==============================] - 0s 616us/step - loss: 0.9110 - accuracy: 0.6021
Epoch 481/500
45/45 [==============================] - 0s 625us/step - loss: 0.9106 - accuracy: 0.6021
Epoch 482/500
45/45 [==============================] - 0s 666us/step - loss: 0.9101 - accuracy: 0.6007
Epoch 483/500
45/45 [==============================] - 0s 669us/step - loss: 0.9097 - accuracy: 0.6000
Epoch 484/500
45/45 [==============================] - 0s 689us/step - loss: 0.9092 - accuracy: 0.6028
Epoch 485/500
45/45 [==============================] - 0s 655us/step - loss: 0.9086 - a

In [44]:
print('Validation accuracy:', 
      accuracy_score(y_val, model(X_val).numpy().argmax(axis=1)))

Validation accuracy: 0.5987394957983193


* Few FC layers:

In [47]:
loss_func = keras.losses.CategoricalCrossentropy(from_logits=True)

lr = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

model = keras.Sequential([
    keras.layers.Dense(units=256, input_shape=[n_feats]),
    keras.layers.ReLU(),
#     keras.layers.Dropout(0.2),
    keras.layers.Dense(units=n_classes)
])
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])

In [48]:
model.training = True
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
45/45 [==============================] - 0s 825us/step - loss: 1.1924 - accuracy: 0.5228
Epoch 2/500
45/45 [==============================] - 0s 766us/step - loss: 1.0964 - accuracy: 0.5270
Epoch 3/500
45/45 [==============================] - 0s 702us/step - loss: 1.0145 - accuracy: 0.5656
Epoch 4/500
45/45 [==============================] - 0s 708us/step - loss: 0.9317 - accuracy: 0.6056
Epoch 5/500
45/45 [==============================] - 0s 699us/step - loss: 0.8613 - accuracy: 0.6428
Epoch 6/500
45/45 [==============================] - 0s 696us/step - loss: 0.8032 - accuracy: 0.6947
Epoch 7/500
45/45 [==============================] - 0s 710us/step - loss: 0.7416 - accuracy: 0.7200
Epoch 8/500
45/45 [==============================] - 0s 706us/step - loss: 0.6778 - accuracy: 0.7628
Epoch 9/500
45/45 [==============================] - 0s 705us/step - loss: 0.6291 - accuracy: 0.7902
Epoch 10/500
45/45 [==============================] - 0s 696us/step - loss: 0.5896 - accura

45/45 [==============================] - 0s 733us/step - loss: 0.2177 - accuracy: 0.9221
Epoch 82/500
45/45 [==============================] - 0s 738us/step - loss: 0.2171 - accuracy: 0.9249
Epoch 83/500
45/45 [==============================] - 0s 729us/step - loss: 0.2023 - accuracy: 0.9305
Epoch 84/500
45/45 [==============================] - 0s 732us/step - loss: 0.2066 - accuracy: 0.9172
Epoch 85/500
45/45 [==============================] - 0s 716us/step - loss: 0.2023 - accuracy: 0.9340
Epoch 86/500
45/45 [==============================] - 0s 743us/step - loss: 0.2051 - accuracy: 0.9312
Epoch 87/500
45/45 [==============================] - 0s 710us/step - loss: 0.2054 - accuracy: 0.9256
Epoch 88/500
45/45 [==============================] - 0s 708us/step - loss: 0.2032 - accuracy: 0.9298
Epoch 89/500
45/45 [==============================] - 0s 740us/step - loss: 0.1940 - accuracy: 0.9375
Epoch 90/500
45/45 [==============================] - 0s 755us/step - loss: 0.2023 - accuracy: 

45/45 [==============================] - 0s 703us/step - loss: 0.1168 - accuracy: 0.9705
Epoch 161/500
45/45 [==============================] - 0s 719us/step - loss: 0.1222 - accuracy: 0.9628
Epoch 162/500
45/45 [==============================] - 0s 750us/step - loss: 0.1204 - accuracy: 0.9621
Epoch 163/500
45/45 [==============================] - 0s 709us/step - loss: 0.1220 - accuracy: 0.9593
Epoch 164/500
45/45 [==============================] - 0s 711us/step - loss: 0.1341 - accuracy: 0.9579
Epoch 165/500
45/45 [==============================] - 0s 707us/step - loss: 0.1164 - accuracy: 0.9628
Epoch 166/500
45/45 [==============================] - 0s 719us/step - loss: 0.1175 - accuracy: 0.9670
Epoch 167/500
45/45 [==============================] - 0s 723us/step - loss: 0.1237 - accuracy: 0.9635
Epoch 168/500
45/45 [==============================] - 0s 723us/step - loss: 0.1182 - accuracy: 0.9614
Epoch 169/500
45/45 [==============================] - 0s 732us/step - loss: 0.1182 - a

45/45 [==============================] - 0s 695us/step - loss: 0.0760 - accuracy: 0.9796
Epoch 240/500
45/45 [==============================] - 0s 722us/step - loss: 0.0787 - accuracy: 0.9775
Epoch 241/500
45/45 [==============================] - 0s 723us/step - loss: 0.0838 - accuracy: 0.9775
Epoch 242/500
45/45 [==============================] - 0s 723us/step - loss: 0.0798 - accuracy: 0.9804
Epoch 243/500
45/45 [==============================] - 0s 757us/step - loss: 0.0800 - accuracy: 0.9775
Epoch 244/500
45/45 [==============================] - 0s 761us/step - loss: 0.0855 - accuracy: 0.9747
Epoch 245/500
45/45 [==============================] - 0s 740us/step - loss: 0.0808 - accuracy: 0.9740
Epoch 246/500
45/45 [==============================] - 0s 723us/step - loss: 0.0825 - accuracy: 0.9768
Epoch 247/500
45/45 [==============================] - 0s 768us/step - loss: 0.0768 - accuracy: 0.9761
Epoch 248/500
45/45 [==============================] - 0s 774us/step - loss: 0.0716 - a

45/45 [==============================] - 0s 711us/step - loss: 0.0629 - accuracy: 0.9804
Epoch 319/500
45/45 [==============================] - 0s 732us/step - loss: 0.0576 - accuracy: 0.9867
Epoch 320/500
45/45 [==============================] - 0s 754us/step - loss: 0.0631 - accuracy: 0.9811
Epoch 321/500
45/45 [==============================] - 0s 727us/step - loss: 0.0723 - accuracy: 0.9789
Epoch 322/500
45/45 [==============================] - 0s 729us/step - loss: 0.0573 - accuracy: 0.9860
Epoch 323/500
45/45 [==============================] - 0s 738us/step - loss: 0.0609 - accuracy: 0.9811
Epoch 324/500
45/45 [==============================] - 0s 734us/step - loss: 0.0499 - accuracy: 0.9825
Epoch 325/500
45/45 [==============================] - 0s 731us/step - loss: 0.0483 - accuracy: 0.9909
Epoch 326/500
45/45 [==============================] - 0s 752us/step - loss: 0.0569 - accuracy: 0.9818
Epoch 327/500
45/45 [==============================] - 0s 739us/step - loss: 0.0494 - a

45/45 [==============================] - 0s 723us/step - loss: 0.0366 - accuracy: 0.9902
Epoch 398/500
45/45 [==============================] - 0s 728us/step - loss: 0.0393 - accuracy: 0.9895
Epoch 399/500
45/45 [==============================] - 0s 720us/step - loss: 0.0347 - accuracy: 0.9923
Epoch 400/500
45/45 [==============================] - 0s 715us/step - loss: 0.0403 - accuracy: 0.9895
Epoch 401/500
45/45 [==============================] - 0s 749us/step - loss: 0.0363 - accuracy: 0.9902
Epoch 402/500
45/45 [==============================] - 0s 724us/step - loss: 0.0360 - accuracy: 0.9916
Epoch 403/500
45/45 [==============================] - 0s 765us/step - loss: 0.0347 - accuracy: 0.9909
Epoch 404/500
45/45 [==============================] - 0s 716us/step - loss: 0.0361 - accuracy: 0.9937
Epoch 405/500
45/45 [==============================] - 0s 728us/step - loss: 0.0393 - accuracy: 0.9909
Epoch 406/500
45/45 [==============================] - 0s 724us/step - loss: 0.0388 - a

45/45 [==============================] - 0s 792us/step - loss: 0.0263 - accuracy: 0.9951
Epoch 477/500
45/45 [==============================] - 0s 696us/step - loss: 0.0246 - accuracy: 0.9965
Epoch 478/500
45/45 [==============================] - 0s 699us/step - loss: 0.0263 - accuracy: 0.9944
Epoch 479/500
45/45 [==============================] - 0s 755us/step - loss: 0.0254 - accuracy: 0.9951
Epoch 480/500
45/45 [==============================] - 0s 717us/step - loss: 0.0291 - accuracy: 0.9944
Epoch 481/500
45/45 [==============================] - 0s 702us/step - loss: 0.0236 - accuracy: 0.9937
Epoch 482/500
45/45 [==============================] - 0s 702us/step - loss: 0.0270 - accuracy: 0.9916
Epoch 483/500
45/45 [==============================] - 0s 731us/step - loss: 0.0216 - accuracy: 0.9958
Epoch 484/500
45/45 [==============================] - 0s 748us/step - loss: 0.0204 - accuracy: 0.9958
Epoch 485/500
45/45 [==============================] - 0s 742us/step - loss: 0.0261 - a

In [49]:
model.training = False
print('Validation accuracy:', 
      accuracy_score(y_val, model(X_val).numpy().argmax(axis=1)))

Validation accuracy: 0.9600840336134454


* Export to tflite:

In [51]:
models_root = './robot_models'
os.makedirs(models_root, exist_ok=True)

name = 'robot_fc256'

keras.models.save_model(model, f'{models_root}/{name}.h5')

# !!! AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'TFLiteConverter'
# converter = tf.compat.v1.TFLiteConverter.from_keras_model_file(keras_file)
# tflite_model = converter.convert()
# open('linear.tflite', 'wb').write(tflite_model)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open(f'{models_root}/{name}.tflite', 'wb') as file:
    file.write(tflite_model)